#### Test file parameters: rho_10_theta_0.01 sample_size_20 depth_4 genome_size_5000 seed_1
#### The steps that follow the generation of a pairwise table:


In [11]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from ldpop import rhos_from_string

import m_isolate_by_depth
import m_biallelic_filter_pairwise_table
import m_pairwise_lookup_format_pyrho
import m_custom_hap_sets_and_merge
import m_pij_grid_vectorised
import m_pairwise_rho_estimator_intp_rect_biv

depth = 10 # Change to try different depths. Needs appropriate lookup tables though

recom_tract_len = 1000
depth_range = "3,200"
n_resamples = 50
lookup_table_rho_range = "101,100"
pairwise_table_file = "../Recom_Est_Output/pairwise_table.pkl"
num_cores = 4
lookup_table_rho_vals = rhos_from_string(lookup_table_rho_range)
lookup_table = f"/Volumes/Backup/Lookup_tables/Lookup_tables_m_0.01_r_0-100/lk_downsampled_{depth}.csv"

#### Load pairwise table

In [12]:
pairwise_table = pd.read_pickle(pairwise_table_file)
pairwise_table

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
"(30, 79)",0,0,0,0,0,0,0,3,0,0,4,3,0,0,0,0
"(30, 100)",0,0,0,0,0,0,3,0,0,0,7,0,0,0,0,0
"(30, 118)",0,0,0,0,1,0,2,0,0,0,7,0,0,0,0,0
"(30, 126)",0,0,0,0,0,0,0,3,0,0,5,2,0,0,0,0
"(30, 168)",0,0,0,0,0,0,0,1,0,0,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(4833, 4909)",0,0,0,6,1,0,0,0,15,0,0,2,0,0,0,0
"(4833, 4935)",0,0,0,5,0,0,0,1,0,2,0,12,0,0,0,0
"(4854, 4909)",8,0,0,0,0,0,0,0,9,0,0,9,0,0,0,0
"(4854, 4935)",0,0,0,6,0,0,0,0,0,2,0,14,0,0,0,0


#### Isolate a single depth for testing

In [13]:
pairwise_table_slice = m_isolate_by_depth.main(pairwise_table, depth)
pairwise_table_slice

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
"(30, 79)",0,0,0,0,0,0,0,3,0,0,4,3,0,0,0,0
"(30, 100)",0,0,0,0,0,0,3,0,0,0,7,0,0,0,0,0
"(30, 118)",0,0,0,0,1,0,2,0,0,0,7,0,0,0,0,0
"(30, 126)",0,0,0,0,0,0,0,3,0,0,5,2,0,0,0,0
"(30, 258)",0,0,0,0,0,0,0,3,7,0,0,0,0,0,0,0
"(79, 353)",0,0,0,0,0,0,0,0,0,0,2,1,0,0,7,0
"(118, 384)",0,2,0,0,0,0,0,0,0,8,0,0,0,0,0,0
"(1188, 1457)",0,0,0,0,0,0,5,0,0,1,4,0,0,0,0,0
"(2586, 2862)",4,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0
"(2586, 2864)",0,2,2,0,0,0,6,0,0,0,0,0,0,0,0,0


#### Perform bi-allelic filtering

In [14]:
pairwise_biallelic_table = m_biallelic_filter_pairwise_table.main(pairwise_table_slice.copy())
pairwise_biallelic_table

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
"(30, 79)",0,0,0,0,0,0,0,3,0,0,4,3,0,0,0,0
"(30, 118)",0,0,0,0,1,0,2,0,0,0,7,0,0,0,0,0
"(30, 126)",0,0,0,0,0,0,0,3,0,0,5,2,0,0,0,0
"(30, 258)",0,0,0,0,0,0,0,3,7,0,0,0,0,0,0,0
"(79, 353)",0,0,0,0,0,0,0,0,0,0,2,1,0,0,7,0
"(1188, 1457)",0,0,0,0,0,0,5,0,0,1,4,0,0,0,0,0
"(2586, 2862)",4,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0
"(2586, 2864)",0,2,2,0,0,0,6,0,0,0,0,0,0,0,0,0
"(2691, 2952)",0,0,0,0,5,0,0,0,3,0,2,0,0,0,0,0
"(2691, 2955)",0,0,0,0,5,0,0,0,2,0,0,3,0,0,0,0


#### Convert to lookup format to match against likelihood tables

In [15]:
lookup_formatted_table = m_pairwise_lookup_format_pyrho.main(pairwise_biallelic_table.copy())
lookup_formatted_table

,00,01,10,11
"(30, 79)",3.0,4.0,3.0,0.0
"(30, 118)",7.0,0.0,2.0,1.0
"(30, 126)",2.0,5.0,3.0,0.0
"(30, 258)",0.0,7.0,3.0,0.0
"(79, 353)",0.0,7.0,1.0,2.0
"(1188, 1457)",4.0,1.0,5.0,0.0
"(2586, 2862)",3.0,3.0,0.0,4.0
"(2586, 2864)",6.0,0.0,2.0,2.0
"(2691, 2952)",2.0,3.0,0.0,5.0
"(2691, 2955)",3.0,2.0,0.0,5.0


#### Merge lookup formatted table on likelihood table

In [16]:
merged_eq3_table, table_ids_for_eq3 = m_custom_hap_sets_and_merge.main(pairwise_biallelic_table.copy(),
                                                                           lookup_formatted_table.copy(),
                                                                           lookup_table_rho_vals,
                                                                           lookup_table)
merged_eq3_table

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.000000000000001,8.0,9.0,...,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,d_ij
0,-21.455557,-21.365481,-21.377219,-21.412870,-21.454733,-21.497122,-21.537956,-21.576479,-21.612479,-21.645982,...,-22.236349,-22.237772,-22.239167,-22.240535,-22.241877,-22.243194,-22.244487,-22.245756,-22.247001,49
1,-18.462363,-18.342202,-18.316289,-18.312179,-18.314559,-18.318862,-18.323511,-18.327942,-18.331969,-18.335553,...,-18.368204,-18.368225,-18.368245,-18.368265,-18.368284,-18.368302,-18.368320,-18.368338,-18.368355,88
2,-21.152079,-20.981607,-21.010753,-21.077759,-21.151990,-21.224735,-21.293249,-21.356801,-21.415407,-21.469362,...,-22.379252,-22.381442,-22.383590,-22.385697,-22.387764,-22.389793,-22.391784,-22.393739,-22.395658,96
3,-15.929185,-16.651006,-17.152080,-17.532082,-17.835323,-18.085725,-18.297703,-18.480596,-18.640782,-18.782795,...,-21.050410,-21.056500,-21.062479,-21.068349,-21.074113,-21.079774,-21.085335,-21.090799,-21.096168,228
4,-18.462363,-18.342202,-18.316289,-18.312179,-18.314559,-18.318862,-18.323511,-18.327942,-18.331969,-18.335553,...,-18.368204,-18.368225,-18.368245,-18.368265,-18.368284,-18.368302,-18.368320,-18.368338,-18.368355,274
5,-19.391391,-19.335923,-19.312769,-19.300741,-19.293770,-19.289459,-19.286679,-19.284837,-19.283595,-19.282750,...,-19.283145,-19.283151,-19.283157,-19.283162,-19.283167,-19.283172,-19.283178,-19.283182,-19.283187,269
6,-21.455557,-21.365481,-21.377219,-21.412870,-21.454733,-21.497122,-21.537956,-21.576479,-21.612479,-21.645982,...,-22.236349,-22.237772,-22.239167,-22.240535,-22.241877,-22.243194,-22.244487,-22.245756,-22.247001,276
7,-20.242070,-20.089156,-20.095516,-20.133641,-20.178574,-20.223213,-20.265331,-20.304321,-20.340169,-20.373074,...,-20.950288,-20.951894,-20.953471,-20.955021,-20.956544,-20.958041,-20.959512,-20.960958,-20.962380,278
8,-20.765645,-20.679014,-20.677527,-20.696906,-20.722255,-20.748744,-20.774583,-20.799094,-20.822059,-20.843459,...,-21.237087,-21.238155,-21.239204,-21.240235,-21.241246,-21.242240,-21.243217,-21.244176,-21.245119,261
9,-21.152079,-20.981607,-21.010753,-21.077759,-21.151990,-21.224735,-21.293249,-21.356801,-21.415407,-21.469362,...,-22.379252,-22.381442,-22.383590,-22.385697,-22.387764,-22.389793,-22.391784,-22.393739,-22.395658,264


#### Calculate p_ij values for variant pairs

In [17]:
p_ij_grid = m_pij_grid_vectorised.main(recom_tract_len, lookup_table_rho_vals, merged_eq3_table.copy())
p_ij_grid

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
0,0.0,0.095638,0.191275,0.286913,0.382551,0.478189,0.573826,0.669464,0.765102,0.860740,...,8.703034,8.798672,8.894310,8.989948,9.085585,9.181223,9.276861,9.372499,9.468136,9.563774
1,0.0,0.168478,0.336956,0.505435,0.673913,0.842391,1.010869,1.179348,1.347826,1.516304,...,15.331520,15.499999,15.668477,15.836955,16.005433,16.173912,16.342390,16.510868,16.679346,16.847825
2,0.0,0.183072,0.366144,0.549216,0.732288,0.915360,1.098432,1.281504,1.464576,1.647648,...,16.659549,16.842621,17.025693,17.208765,17.391837,17.574909,17.757981,17.941053,18.124125,18.307197
3,0.0,0.407751,0.815503,1.223254,1.631006,2.038757,2.446509,2.854260,3.262012,3.669763,...,37.105385,37.513136,37.920888,38.328639,38.736391,39.144142,39.551894,39.959645,40.367397,40.775148
4,0.0,0.479336,0.958672,1.438008,1.917343,2.396679,2.876015,3.355351,3.834687,4.314023,...,43.619562,44.098898,44.578234,45.057570,45.536906,46.016242,46.495577,46.974913,47.454249,47.933585
5,0.0,0.471713,0.943427,1.415140,1.886854,2.358567,2.830281,3.301994,3.773708,4.245421,...,42.925927,43.397641,43.869354,44.341068,44.812781,45.284495,45.756208,46.227922,46.699635,47.171349
6,0.0,0.482374,0.964748,1.447122,1.929497,2.411871,2.894245,3.376619,3.858993,4.341367,...,43.896047,44.378421,44.860795,45.343169,45.825543,46.307917,46.790291,47.272666,47.755040,48.237414
7,0.0,0.485406,0.970813,1.456219,1.941625,2.427032,2.912438,3.397844,3.883251,4.368657,...,44.171978,44.657385,45.142791,45.628198,46.113604,46.599010,47.084417,47.569823,48.055229,48.540636
8,0.0,0.459438,0.918876,1.378314,1.837753,2.297191,2.756629,3.216067,3.675505,4.134943,...,41.808873,42.268311,42.727749,43.187187,43.646625,44.106063,44.565502,45.024940,45.484378,45.943816
9,0.0,0.464053,0.928106,1.392159,1.856212,2.320265,2.784318,3.248370,3.712423,4.176476,...,42.228816,42.692869,43.156922,43.620975,44.085027,44.549080,45.013133,45.477186,45.941239,46.405292


#### Get final pairwise (variant pairs) likelihoods

In [18]:
interpolated_eq2_df = m_pairwise_rho_estimator_intp_rect_biv.main(merged_eq3_table.copy(),
                                                                      table_ids_for_eq3.copy(),
                                                                      p_ij_grid.copy(),
                                                                      lookup_table,
                                                                      depth)
interpolated_eq2_df

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
0,-21.455557,-21.446942,-21.438328,-21.429713,-21.421098,-21.412484,-21.403869,-21.395254,-21.386639,-21.378025,...,-21.636033,-21.639237,-21.642441,-21.645645,-21.648646,-21.651623,-21.654601,-21.657578,-21.660555,-21.663532
1,-18.462363,-18.442118,-18.421874,-18.401629,-18.381385,-18.361140,-18.341920,-18.337554,-18.333189,-18.328823,...,-18.350220,-18.350474,-18.350728,-18.350982,-18.351235,-18.351462,-18.351688,-18.351915,-18.352142,-18.352369
2,-21.152079,-21.120870,-21.089662,-21.058453,-21.027244,-20.996036,-20.984476,-20.989811,-20.995147,-21.000483,...,-21.767598,-21.772990,-21.778333,-21.783378,-21.788423,-21.793468,-21.798514,-21.803559,-21.808391,-21.813121
3,-15.929185,-16.223509,-16.517833,-16.762873,-16.967187,-17.166808,-17.321754,-17.476700,-17.611535,-17.735182,...,-20.365523,-20.375673,-20.385823,-20.395677,-20.405459,-20.415119,-20.424554,-20.433990,-20.443129,-20.452235
4,-18.462363,-18.404765,-18.347168,-18.330852,-18.318431,-18.314659,-18.312689,-18.313025,-18.314166,-18.315910,...,-18.365265,-18.365339,-18.365410,-18.365481,-18.365548,-18.365614,-18.365678,-18.365741,-18.365801,-18.365861
5,-19.391391,-19.365226,-19.339061,-19.326311,-19.315389,-19.308456,-19.302782,-19.298636,-19.295347,-19.292712,...,-19.282439,-19.282454,-19.282468,-19.282483,-19.282497,-19.282510,-19.282524,-19.282537,-19.282550,-19.282563
6,-21.455557,-21.412107,-21.368656,-21.370729,-21.376391,-21.391902,-21.409100,-21.428637,-21.448830,-21.469204,...,-22.105697,-22.108126,-22.110537,-22.112887,-22.115213,-22.117489,-22.119735,-22.121938,-22.124109,-22.126243
7,-20.242070,-20.167844,-20.093619,-20.092057,-20.095144,-20.111797,-20.130303,-20.151517,-20.173328,-20.195030,...,-20.813371,-20.815812,-20.818230,-20.820594,-20.822940,-20.825229,-20.827507,-20.829726,-20.831937,-20.834089
8,-20.765645,-20.725843,-20.686042,-20.678451,-20.677768,-20.683286,-20.692190,-20.702383,-20.714029,-20.725829,...,-21.135843,-21.137565,-21.139261,-21.140934,-21.142573,-21.144200,-21.145785,-21.147367,-21.148901,-21.150434
9,-21.152079,-21.072971,-20.993863,-20.993037,-21.006562,-21.032212,-21.063307,-21.096196,-21.130643,-21.164828,...,-22.166007,-22.169805,-22.173556,-22.177217,-22.180854,-22.184386,-22.187914,-22.191323,-22.194732,-22.198040


#### Apply weighting technique
Step 1: Convert log-likelihoods to likelihoods

In [23]:
interpolated_eq2_np = interpolated_eq2_df.to_numpy(dtype=np.float128, copy=True)
exp_np = np.exp(interpolated_eq2_np)
exp_np

array([[4.80806137e-10, 4.84966026e-10, 4.89161906e-10, ...,
        3.92856080e-10, 3.91688206e-10, 3.90523803e-10],
       [9.59174940e-09, 9.78790834e-09, 9.98807889e-09, ...,
        1.07118503e-08, 1.07094214e-08, 1.07069932e-08],
       [6.51281571e-10, 6.71927714e-10, 6.93228356e-10, ...,
        3.39496060e-10, 3.37859639e-10, 3.36265188e-10],
       ...,
       [9.59174940e-09, 1.00734078e-08, 1.05792530e-08, ...,
        1.05765078e-08, 1.05756950e-08, 1.05748874e-08],
       [3.78813758e-09, 3.88757308e-09, 3.98961869e-09, ...,
        4.22382579e-09, 4.22377064e-09, 4.22371659e-09],
       [3.78813758e-09, 3.88955093e-09, 3.99367925e-09, ...,
        4.22372071e-09, 4.22366631e-09, 4.22361343e-09]], dtype=float128)

Step 2: We normalise the likelihood values such that the values add up to 1. We do this by dividing each value by the sum of all values.

In [24]:
exp_norm_np = exp_np / np.sum(exp_np)
exp_norm_np

array([[5.72716450e-06, 5.77671538e-06, 5.82669497e-06, ...,
        4.67953968e-06, 4.66562844e-06, 4.65175856e-06],
       [1.14252965e-04, 1.16589529e-04, 1.18973878e-04, ...,
        1.27595144e-04, 1.27566213e-04, 1.27537289e-04],
       [7.75779760e-06, 8.00372594e-06, 8.25745041e-06, ...,
        4.04393712e-06, 4.02444474e-06, 4.00545230e-06],
       ...,
       [1.14252965e-04, 1.19990281e-04, 1.26015701e-04, ...,
        1.25983001e-04, 1.25973320e-04, 1.25963700e-04],
       [4.51227332e-05, 4.63071680e-05, 4.75226932e-05, ...,
        5.03124716e-05, 5.03118147e-05, 5.03111709e-05],
       [4.51227332e-05, 4.63307273e-05, 4.75710609e-05, ...,
        5.03112200e-05, 5.03105720e-05, 5.03099421e-05]], dtype=float128)

Step 3: Multiply the values by the depth and the number of observations. This scaling is the “weighting” procedure, the higher the number of observations or depth the more it will be weighted for the final log sum calculation.

In [31]:
observations = exp_norm_np.shape[0]
exp_norm_scaled_np = depth * observations * exp_norm_np
exp_norm_scaled_np

array([[0.00085907, 0.00086651, 0.000874  , ..., 0.00070193, 0.00069984,
        0.00069776],
       [0.01713794, 0.01748843, 0.01784608, ..., 0.01913927, 0.01913493,
        0.01913059],
       [0.00116367, 0.00120056, 0.00123862, ..., 0.00060659, 0.00060367,
        0.00060082],
       ...,
       [0.01713794, 0.01799854, 0.01890236, ..., 0.01889745, 0.018896  ,
        0.01889455],
       [0.00676841, 0.00694608, 0.0071284 , ..., 0.00754687, 0.00754677,
        0.00754668],
       [0.00676841, 0.00694961, 0.00713566, ..., 0.00754668, 0.00754659,
        0.00754649]], dtype=float128)

Step 4: Following this weighting step, we convert back to log space by taking the log of the values

In [33]:
weighted_log_np = np.log(exp_norm_scaled_np)
weighted_log_np

array([[-7.05965471, -7.05104002, -7.04242532, ..., -7.26167552,
        -7.26465272, -7.26762993],
       [-4.06646028, -4.04621579, -4.0259713 , ..., -3.95601295,
        -3.95623971, -3.95646648],
       [-6.75617678, -6.72496809, -6.69375939, ..., -7.40765651,
        -7.41248832, -7.41721876],
       ...,
       [-4.06646028, -4.01746452, -3.96846875, ..., -3.96872828,
        -3.96880513, -3.96888149],
       [-4.99548908, -4.9695785 , -4.94366792, ..., -4.88662227,
        -4.88663533, -4.88664813],
       [-4.99548908, -4.96906987, -4.94265065, ..., -4.88664715,
        -4.88666003, -4.88667255]], dtype=float128)

#### ...Collect pairwise likelihoods across depths, bootstrap and perform final sums
Here we are only looking at one depth